In [1]:
# imports 
%load_ext autoreload
%autoreload 2
import schillinger.rhythm as SR
import schillinger.pitch as SP
import itertools
import random
from music21 import *
%load_ext music21.ipython21

In [2]:
#convert2d to 1d 
def convert2Dto1D(data):
    flat_array = []
    for d in data:
        if isinstance(d[0], list) or isinstance(d[0], tuple):
            for l in d:
                
                flat_array.append(l)
        else:
            flat_array.append(d)
    return flat_array

In [12]:
# notes harmony 

def arpeggiator(c, es,rhythm, duration_divider):
    chords = []
    for i, seq in enumerate(c):
                
        nota = seq
        nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
        chords.append(nota)

    nota = pitch.Pitch(c.pitches[0])
    
    
    nota.octave = nota.octave + 2
    nota = note.Note(nota.nameWithOctave)
    #nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    
   
    chords.append(nota)
    #octave adjust
    
    
    #print(chords[-1].pitch.midi - chords[-2].pitch.midi, chords[-1])
    if chords[-1].pitch.midi - chords[-2].pitch.midi >= 11:
        chords[-1].octave = chords[-1].octave -1
        #print("yes")
    ##
    nota = pitch.Pitch(c.pitches[2])
    nota.octave = nota.octave
    nota = note.Note(nota.nameWithOctave)
    #nota.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    return chords

def showHarmony(note_sequence, scala, rhythm, length, time_meter, duration_divider, arp=False,bass=True, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='melody')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='chords')
    p1.timeSignature = meter.TimeSignature(TS)
    p2 = stream.Part(id='bass')
    p2.timeSignature = meter.TimeSignature(TS)
    
    #pseudo stuff
    temp_scale = [str(p) for p in scala.getPitches('C2', 'C6')] # for pseudo melody (scale.next bug!)
    rhythm_seq = [2,1,1,2]
    for es, sequences in enumerate(note_sequence):
        
        melody = []
        chords = []
        bass = []
        
        
        ### chords
        #chord_seq = [sequences[0]]+sequences[1]
        chord_seq = sequences
        #print(chord_seq)
        test_chord = chord.Chord(chord_seq)
        c = chord.Chord(test_chord)
        c.root = sequences[0]
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        c = c.sortDiatonicAscending() # for better voice leadng but not real!
        if es % 2 == 0: # rotating clock counterclock positions
            c = c.semiClosedPosition(inPlace=False,forceOctave=4)
            pass
        else:
            c = c.closedPosition(inPlace=False,forceOctave=4)
            pass
        if drop2:
            try:
                c.pitches[0].octave = c.pitches[0].octave-1 # drop2
                #c.pitches[-2].octave = c.pitches[-2].octave+2 # drop4
                pass
            except:
                pass
        
        if arp:
            chords =  arpeggiator(c, es, rhythm, duration_divider)
            #chords.append(nota)
            
        else:
            chords.append(c)
            
        ##############
        # pseudo melody
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(dur)
        
        #check to make long or short notes.
        if random.choice([True,False]) or es % 8 == 0:
            t_nota = pitch.Pitch(random.choice(sequences))
            #nota.octave = nota.octave
            nota = note.Note(t_nota.nameWithOctave)
            nota.duration.quarterLength = dur
            melody.append(nota)
        else:
            t_nota = pitch.Pitch(random.choice(sequences))
            nota = note.Note(t_nota.nameWithOctave)
            nota.duration.quarterLength = dur/2
            melody.append(nota)
            #get next tone in scale -1 or +1 for 2nd note
            t_nota_idx = temp_scale.index(t_nota.nameWithOctave) + random.choice([-1,1])
            t_nota = pitch.Pitch(temp_scale[t_nota_idx])
            nota = note.Note(t_nota.nameWithOctave)
            nota.duration.quarterLength = dur/2
            melody.append(nota)
            
        ##############
        # pseudo BASS
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(dur)
        
        #check to make long or short notes.
        if random.choice([True,True,False]):
            t_nota = pitch.Pitch(sequences[0])
            
            nota = note.Note(t_nota.nameWithOctave)
            nota.octave = nota.octave - 2
            nota.duration.quarterLength = dur
            bass.append(nota)
        else:
            t_nota = pitch.Pitch(sequences[0])
            nota = note.Note(t_nota.nameWithOctave)
            nota.octave = nota.octave - 2
            nota.duration.quarterLength = dur/2
            bass.append(nota)
            #get next tone in scale -1 or +1 for 2nd note
            t_nota_idx = temp_scale.index(t_nota.nameWithOctave) + random.choice([-1,1])
            t_nota = pitch.Pitch(temp_scale[t_nota_idx])
            nota = note.Note(t_nota.nameWithOctave)
            nota.octave = nota.octave - 2
            nota.duration.quarterLength = dur/2
            bass.append(nota)
            
            
        p0.append(melody)
        p1.append(chords)
        p2.append(bass)
    
    s.insert(0, p0)
    s.insert(0, p1)
    s.insert(0, p2)
    #s = s.transpose(0)
    return s



In [4]:
# SCALES 
hm = scale.HarmonicMinorScale("C4")
do = scale.DorianScale("C4")
ms = scale.MixolydianScale("C4")
ls = scale.LydianScale("C4")
lks = scale.LocrianScale('C4')

main_scale = hm

In [5]:
# BASS OR MELODY! ->

In [14]:
# BASS Diatonic Harmony Cycles 

def harmonize_dt_bass(scales, diatonic_cycles_string, coeficients):
    SPG = SP.PitchGroup()
    bass = []
    harmonies = []
    for s in scales:
        temp_scale = [str(p) for p in s.pitches[:-1]]
        scale_expansion = SPG.expansions(temp_scale)
        #print(scale_expansion)
        C7 = scale_expansion[0]
        C_7 = scale_expansion[5]
        C3 = scale_expansion[4]
        C_3 = scale_expansion[1]
        C5 = scale_expansion[2]
        C_5 = scale_expansion[3]
        diatonic_cycles = []
        
        for i in diatonic_cycles_string:
            if i == "C3":
                diatonic_cycles.append(C3)
            elif i == "C-3":
                diatonic_cycles.append(C_3)
            elif i == "C5":
                diatonic_cycles.append(C5)
            elif i == "C-5":
                diatonic_cycles.append(C_5)
            elif i == "C7":
                diatonic_cycles.append(C7)
            elif i == "C-7":
                diatonic_cycles.append(C_7)
            else:
                print("diatonic_cycles_string Value Error!")
                
        diatonic_cycle = SP.pitch_cicles(temp_scale, diatonic_cycles, coeficients)
        harmonies.append(temp_scale)
        bass.append(diatonic_cycle)
    return [bass, harmonies]

hb = harmonize_dt_bass([main_scale], ["C3","C7","C5","C7","C-3","C5"], [1,1,1,1,1,3])
bass = hb[0]
scales = hb[1]
seq_rhythm = [1]
#print(bass,scales)
#[str(p) for p in hm.getPitches('C3', 'C6')]
#hm.next("B3", direction=hm.DIRECTION_ASCENDING)
print(hb[0], hb[1])
#sum([1,1,1,1,1,3])*len(["C3","C7","C5","C7","C-3","C5"])

[['C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4', 'A-4', 'B4', 'E-4', 'F4', 'A-4', 'D4', 'G4', 'C4']] [['C4', 'D4', 'E-4', 'F4', 'G4', 'A-4', 'B4']]


In [9]:
# MELODY Diatonic Harmony Cycles 
melody = [["G","E-","F","G","C","F","E-","D","E-"]] # bach

main_scale = hm
temp_scale = [str(p.name) for p in main_scale.pitches[:-1]]
scales = [temp_scale]
print(scales)

[['C', 'D', 'E-', 'F', 'G', 'A-', 'B']]


In [17]:
note_seq = melody
print(note_seq)
#note_seq = bass
voice_amount = 4 # for now!
theme_expansion = 2
scale_expansion_amount = 1 # # 1st order expansion for harmonizing is according to schillingers default
whole_seq = SP.get_whole_sequence_canonical(note_seq, scales, voice_amount, theme_expansion, scale_expansion_amount)
merged_seq = list(itertools.chain(*whole_seq))
print(whole_seq,merged_seq,scales)

[['G', 'E-', 'F', 'G', 'C', 'F', 'E-', 'D', 'E-']]


ValueError: 'G' is not in list

In [15]:
# music21 converter 

########################################################
#notes_permutations = SPG.melodic_forms_definition(notes)
#######

s = showHarmony(merged_seq, main_scale, seq_rhythm, len(merged_seq)*2, (3,4), 2, arp=True, bass=False, drop2=True)

s.show('musicxml')
#seq_rhythm

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [16]:
def arranger(note_sequence,time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    #make parts on voice lenght
    
    my_parts = [[]for _ in range(len(note_sequence[0]))]
    #print(len(note_sequence[0]))
    
    for es, sequences in enumerate(note_sequence):
        #print("-"*50)
        #main func
        new_list = sequences[:]
        
        #SWAP!
        
        if es > 0:
            prev_list = note_sequence[es-1]
            for it, nota in enumerate(new_list):
                for idx, pl in enumerate(prev_list):
                    if pl == nota:
                        source_pitch = new_list[it].pitch
                        target_pitch = new_list[idx].pitch
                        new_list[idx].pitch = source_pitch
                        new_list[it].pitch = target_pitch
        
        #print(new_list)
        
        ####
        for n, nota in enumerate(new_list):
            if es == 0:
                #print("append all", nota, nota.quarterLength)
                my_parts[n].append(nota)    
            else:
                if my_parts[n][-1].nameWithOctave != nota.nameWithOctave:
                    my_parts[n].append(nota)
                else:
                    my_parts[n][-1].tie = tie.Tie("start")
                    my_parts[n].append(nota)
                    
                pass
    
    parts = []
    for p in range(len(note_sequence[0])): 
        parts.append(stream.Part(id=p))
        parts[p].timeSignature = meter.TimeSignature(TS)
    
    for it, p in enumerate(my_parts):
        parts[it].append(p)
        
    for i in parts:
        s.insert(0, i)
    return s

def arranger_simple(note_sequence, durations,time_meter, duration_divider):
    
    chords = []
    
    for es, sequences in enumerate(note_sequence):
        
        c = chord.Chord(sequences)
        c.root = sequences[0]
        c.duration.quarterLength = durations[es%len(durations)]*duration_divider
        c.pitches[0].octave = c.pitches[0].octave-1 # drop2
        chords.append(c)
        
    return arranger(chords, time_meter, duration_divider)

    '''for i in parts:
        s.insert(0, i)
        
    
    return s'''

durs = [2,1,1,2]
s = arranger_simple(merged_seq, durs,(4,4), 2)
s.show('musicxml')

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'